In [1]:
import pandas as pd


sbc = pd.read_csv("extdata/sbcdata.csv")

In [ ]:
sbc.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def apply(group):
#     print(np.any(group["Diagnosis"].str.contains("Sepsis").values))
#     if group.shape[0] < 2 or not np.any(group["Diagnosis"].str.contains("Sepsis").values):
#         return
#     group = group.sort_values("Time")
#     plt.plot(group["Time"], group["WBC"])
#     plt.show()
    return group.shape[0]

sbc_grouped = sbc.groupby("Id").apply(apply)

In [ ]:
sbc_grouped = sbc.groupby("Id").filter(lambda x: x.shape[0] >= 10)

In [ ]:
sbc_grouped[sbc_grouped["Diagnosis"] == "Sepsis"]

In [ ]:
sbc_ones = sbc.groupby("Id").filter(lambda x: x.shape[0] == 1)

In [ ]:
sbc_ones_sepsis = sbc_ones.loc[sbc_ones["Diagnosis"] == "Sepsis", :]
sbc_ones_sepsis_val = sbc_ones_sepsis.loc[sbc_ones["Set"] == "Validation", :]
sbc_ones_sepsis_val

In [ ]:
sbc.groupby("Id").apply(lambda x: x.loc[x["Diagnosis"] == "Sepsis", :])

In [1]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data, None)

/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [2]:
from dataAnalysis.Constants import FEATURES_IN_TABLE
import numpy as np

train_data = data_analysis.get_training_data()
test_data = data_analysis.get_testing_data()
test_gw_data = data_analysis.get_gw_testing_data()
WINDOW_SIZE = 5

def get_seq(data):
    data["Label"] = data.loc[:, "Label"] == "Sepsis"
    data["Sex"] = data.loc[:, "Sex"] == "W"
    data = data.astype({'Sex': 'int8', "Label": "int8"})
    features = []
    labels = []
    for identifier, group in data.groupby(["Id","Center"]):
        sorted_group = group.sort_values("Time", ascending=False)
        if sorted_group.shape[0] < WINDOW_SIZE:
            first_element = sorted_group.iloc[0, :]
            first_features = np.expand_dims(first_element.loc[FEATURES_IN_TABLE].values, 0)
            first_seq = np.repeat(first_features, WINDOW_SIZE - sorted_group.shape[0], axis= 0)
            second_seq = sorted_group.loc[:, FEATURES_IN_TABLE].values
            features.append(np.concatenate((first_seq, second_seq), axis = 0))
            labels.append(sorted_group.iloc[-1]["Label"])
        if sorted_group.shape[0] >= WINDOW_SIZE:
            last_five_elements = sorted_group.iloc[-5:, :]
            last_five_features = last_five_elements.loc[:, FEATURES_IN_TABLE].values
            features.append(last_five_features)
            labels.append(sorted_group.iloc[-1]["Label"])
    return features, labels
            
    
train_features, train_labels = get_seq(train_data)

In [35]:
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch
import math
import os

class PositionalEncoding(nn.Module):

    def __init__(self, input_dim: int, dropout: float = 0.0, max_len: int = 5):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, input_dim, 2) * (-math.log(10000.0) / input_dim))
        pe = torch.zeros(max_len, 1, input_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        print(x)
        x = x + self.pe[:x.size(0)]
        print(x)
        return self.dropout(x)

class Transformer_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_heads = num_heads
        
        self.pos_encoder = PositionalEncoding(input_dim)
        encoder_layers = TransformerEncoderLayer(input_dim, num_heads, hidden_dim, 0)
        self.transformer_encoder = TransformerEncoder(encoder_layers, 2)
        self.linear = nn.Linear(hidden_dim, output_dim)


In [36]:
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from dataAnalysis.Constants import FEATURES

EPOCHS = 100
LEARNINLEARNING_RATERATE = 3e-2

model = Transformer_Model(input_dim = len(FEATURES),hidden_dim = 128, output_dim = 1, num_heads = 1)
optimizer = Adam(model.params, lr = LEARNING_RATE)
loss_fn = BCEWithLogitsLoss(pos_weight=torch.tensor(10))


for epoch in range(EPOCHS):
    model.train()
    out = model(X_train)
    loss = loss_fn(out.unsqueeze(1), y_train)
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: The expanded size of the tensor (7) must match the existing size (4) at non-singleton dimension 2.  Target sizes: [5, 1, 7].  Tensor sizes: [5, 4]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
import numpy as np
import math

# Generate sample time series data
data = torch.tensor(train_features, dtype=torch.float32) #np.random.randn(10000, 5, 7)  # Shape: [10000, 5, 7]
labels = torch.tensor(train_labels) #np.random.randint(2, size=10000)  # Binary labels

# Convert data and labels to PyTorch tensors
data_tensor = data #torch.tensor(data, dtype=torch.float32)
labels_tensor = labels #torch.tensor(labels)

# Create a custom dataset
dataset = TensorDataset(data_tensor, labels_tensor)

# Define the TransformerEncoder class
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads, dropout, max_len=5000):
        super(TransformerEncoder, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim, max_len=max_len)
        encoder_layer = nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=hidden_dim, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Linear(hidden_dim, 1)  # Binary classification

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = x.permute(1, 0, 2)  # Adjust the dimensions for the Transformer
        output = self.transformer_encoder(x)
        output = output.mean(dim=0)  # Average the encoder outputs over the sequence length
        output = self.classifier(output).squeeze(1)

        return output


# Set the device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Instantiate the model
input_dim = 7
hidden_dim = 64
num_layers = 1
num_heads = 1
dropout = 0.1
max_len = data.shape[1]
model = TransformerEncoder(input_dim, hidden_dim, num_layers, num_heads, dropout, max_len=max_len).to(device)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 3
model.train()
for epoch in range(num_epochs):
    total_loss = 0.0

    for i in range(len(dataset)):
        inputs, labels = dataset[i]
        inputs = inputs.unsqueeze(0).to(device)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(0).to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print the average loss for the epoch
    average_loss = total_loss / len(dataset)
    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

# Evaluation (optional)
model.eval()
with torch.inference_mode():
    total_correct = 0
    for i in range(len(dataset)):
        inputs, labels = dataset[i]
        inputs = inputs.unsqueeze(0).to(device)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(0).to(device)

        outputs = model(inputs)
        predicted_labels = (outputs >= 0.5).float()

        total_correct += (predicted_labels.squeeze() == labels.squeeze()).sum().item()

    accuracy = total_correct / len(dataset)
    print(f"Accuracy: {accuracy}")


/tmp/ipykernel_646930/4032326726.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(0).to(device)


In [45]:
import time

for i in range(60):
    print("Sleeping")
    time.sleep(60)

Hi


KeyboardInterrupt: 

## GAT

In [1]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data, None)

/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [1]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [2]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
data = cudf.from_pandas(data)

In [11]:
data.groupby(["Id", "Center"]).apply(lambda x: x.index)

/home/dwalke/.local/lib/python3.10/site-packages/cudf/core/groupby/groupby.py:1191: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 708595 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(


Int64Index([601702,   5962, 756765, 165039,  68017, 832715, 889231, 264347,
            321489, 446789,
            ...
             25563,  62523, 187771,  48581, 151065, 206792, 214540, 222910,
            319258, 364398],
           dtype='int64', length=1381358)

In [6]:
unique_ids = data["Id"].unique()

In [8]:
import cupy as cp
import time

source_edge_index = cp.array([], dtype= cp.int32)
target_edge_index = cp.array([], dtype= cp.int32)

start = time.time()
for i in range(unique_ids.shape[0]):
    Id = unique_ids[i]
    group = data[data["Id"] == Id]
    indices = group.index
    indices = cp.expand_dims(indices.values, axis = 1)
    target = cp.repeat(indices, indices.shape[0], axis = 1)
    source = target.transpose().flatten()
    target = target.flatten()
    
    source_edge_index = cp.concatenate((source_edge_index, source))
    target_edge_index = cp.concatenate((target_edge_index, target))
    if i % 1000 == 0:
        print(f"{str(i / unique_ids.shape[0] * 100)} %")
        print(time.time() - start )
print(source_edge_index.shape)
print(target_edge_index.shape)

0.0 %
0.3863487243652344
0.1411243375976404 %
8.447920322418213
0.2822486751952808 %
15.594621896743774
0.4233730127929212 %
22.813016653060913
0.5644973503905616 %
30.06106734275818
0.705621687988202 %
37.329527854919434
0.8467460255858424 %
44.63788938522339
0.9878703631834829 %
52.003878116607666
1.1289947007811232 %
59.41252279281616
1.2701190383787637 %
66.85702896118164
1.411243375976404 %
74.34396123886108
1.5523677135740444 %
81.85710644721985
1.6934920511716849 %
89.3711051940918
1.8346163887693252 %
96.92373657226562
1.9757407263669657 %
104.50927758216858
2.116865063964606 %
112.10162711143494
2.2579894015622464 %
119.73893761634827
2.399113739159887 %
127.37569284439087
2.5402380767575274 %
135.02311968803406
2.6813624143551675 %
142.7527310848236
2.822486751952808 %
150.48534655570984
2.963611089550448 %
158.2176558971405
3.1047354271480887 %
166.01249814033508
3.2458597647457297 %
173.86647725105286
3.3869841023433698 %
181.71287417411804
3.52810843994101 %
189.5616703033

29.77723523310212 %
1820.773567199707
29.918359570699764 %
1829.7692749500275
30.05948390829741 %
1838.758261680603
30.200608245895044 %
1847.7519180774689
30.341732583492686 %
1856.7575011253357
30.482856921090328 %
1865.7543542385101
30.623981258687966 %
1874.7644464969635
30.765105596285608 %
1883.7565517425537
30.90622993388325 %
1892.7518498897552
31.047354271480888 %
1901.7475593090057
31.18847860907853 %
1910.7353291511536
31.32960294667617 %
1919.737226486206
31.47072728427381 %
1928.751981973648
31.611851621871452 %
1937.7622175216675
31.752975959469094 %
1946.7787611484528
31.89410029706673 %
1955.7873392105103
32.035224634664374 %
1964.8011636734009
32.17634897226201 %
1973.8125562667847
32.31747330985965 %
1982.8322396278381
32.458597647457296 %
1991.8533148765564
32.59972198505493 %
2000.8740351200104
32.74084632265257 %
2009.8771817684174
32.88197066025022 %
2018.8705615997314
33.02309499784785 %
2027.8794844150543
33.164219335445495 %
2036.900752544403
33.30534367304313 

59.55447046620424 %
3912.9282672405243
59.69559480380189 %
3924.600604057312
59.83671914139953 %
3936.290673971176
59.97784347899717 %
3947.997723340988
60.11896781659482 %
3959.6997904777527
60.26009215419245 %
3971.522809267044
60.40121649179009 %
3983.5122632980347
60.54234082938773 %
3995.49862909317
60.68346516698537 %
4007.47674536705
60.82458950458302 %
4019.4675765037537
60.965713842180655 %
4031.467874765396
61.106838179778286 %
4043.4612929821014
61.24796251737593 %
4055.2951600551605
61.38908685497357 %
4066.991245031357
61.530211192571215 %
4078.6760733127594
61.67133553016886 %
4090.418986082077
61.8124598677665 %
4102.1044199466705
61.95358420536413 %
4113.755223989487
62.094708542961776 %
4125.415725708008
62.235832880559414 %
4136.669734239578
62.37695721815706 %
4147.795383214951
62.5180815557547 %
4158.944816112518
62.65920589335234 %
4170.075712680817
62.800330230949974 %
4181.207421064377
62.94145456854762 %
4192.483726024628
63.08257890614526 %
4203.891051769257
63

90.17845172489221 %
6374.763583660126
90.31957606248986 %
6383.413861274719
90.4607004000875 %
6392.06040596962
90.60182473768513 %
6400.7113127708435
90.74294907528278 %
6409.365633249283
90.88407341288041 %
6418.0243372917175
91.02519775047806 %
6426.677649974823
91.1663220880757 %
6435.333281040192
91.30744642567335 %
6443.996109485626
91.44857076327098 %
6452.66317152977
91.58969510086862 %
6461.321263074875
91.73081943846626 %
6469.972707986832
91.8719437760639 %
6478.63169836998
92.01306811366155 %
6487.287954330444
92.15419245125919 %
6495.9322764873505
92.29531678885682 %
6504.6055381298065
92.43644112645445 %
6513.254794597626
92.5775654640521 %
6521.90890455246
92.71868980164975 %
6530.56166267395
92.85981413924739 %
6539.216823339462
93.00093847684504 %
6547.872424125671
93.14206281444267 %
6556.540548086166
93.2831871520403 %
6565.203494310379
93.42431148963794 %
6573.8661522865295
93.56543582723559 %
6582.522985696793
93.70656016483323 %
6591.182771444321
93.84768450243087

In [11]:
print(source_edge_index),
target_edge_index

[     0 147962 552963 ... 366277 366279 366283]


array([     0,      0,      0, ..., 366277, 366279, 366283])

In [ ]:
import numpy as np
import cupy as cp
import time

df_c = data # data_analysis.get_training_data() ## TODO in future merge with test and then split again (train test split via masks or train test split based on timeseries subgraphs)
# source_edge_index_c = cp.array([], dtype= cp.int32)
# target_edge_index_c = cp.array([], dtype= cp.int32)

start = time.time()
i = 0
for Id, group in df.groupby("Id"):
    indices = group.index
    indices = cp.expand_dims(indices.values, axis = 1)
    target = cp.repeat(indices, indices.shape[0], axis = 1)
    source = target.transpose().flatten()
    target = target.flatten()
    if Id == 520861:
        print(Id)
        print(indices)
        break
#     source_edge_index_c = cp.concatenate((source_edge_index_c, source))
#     target_edge_index_c = cp.concatenate((target_edge_index_c, target))
    i+=1
    if i % 1000 == 0:
        print(f"{str(i / unique_ids.shape[0] * 100)} %")
#     if indices.shape[0] >= 100:
#         print(time.time() - start )
#         break
print(time.time() - start )
print(source_edge_index_c.shape)
print(target_edge_index_c.shape)


In [19]:
source_edge_index, target_edge_index

(array([601702,   5962, 756765, ..., 222910, 319258, 364398]),
 array([601702,   5962, 756765, ..., 364398, 364398, 364398]))

In [44]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES
df[SEX_CATEGORY_COLUMN_NAME] = df.loc[:, SEX_COLUMN_NAME] =="W"

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label,SexCategory
index,,,,,,,,,,,,,,,,,,,,
1211783,210439,39,W,Control,Leipzig,Training,AMB,1,0.0,<NA>,<NA>,<NA>,6.9,88.1,<NA>,266.0,3.70,12.0,Control,True
1042196,145178,59,M,Sepsis,Leipzig,Training,GEN,1,3021360.0,MICU,687780.0,105.05,6.5,82.3,<NA>,60.0,3.72,8.0,Control,False


In [188]:
n = 15
m = 6941950
source_edge_index[m:m+n], target_edge_index[m:m+n]

(array([785479, 365813,  94876, 319392,  94876, 319392, 158886, 131094,
        102747,  50835, 264336, 270433, 276166, 340454,  50835]),
 array([785479, 365813,  94876,  94876, 319392, 319392, 158886, 131094,
        102747,  50835,  50835,  50835,  50835,  50835, 264336]))

In [ ]:
#df.iloc[[319258,364398], :]
#df.loc[:, [df.index == 364398]]
df.reset_index()[df.reset_index().index == 264336]

In [214]:
df[df["Id"] == 520861]

,index,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,...,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label,SexCategory
115190,2017884,520861,63,W,Control,Leipzig,Training,GEN,1,531300.0,...,<NA>,6.13,7.1,84.6,<NA>,217.0,4.08,7.7,Control,True
264336,2017885,520861,63,W,Control,Leipzig,Training,GEN,1,709800.0,...,<NA>,17.03,6.0,85.1,<NA>,283.0,3.42,12.3,Control,True
635145,2017883,520861,63,W,Control,Leipzig,Training,GEN,1,0.0,...,<NA>,23.42,7.0,86.8,<NA>,261.0,4.01,10.7,Control,True


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P

P = getPositionEncoding(seq_len=4, d=len(FEATURES))
print(P)

In [ ]:
def ratio_bool_switch(tensor, ratio = 0.8):
    random = np.random.uniform(0, 1 ,tensor.shape[0])
    val_mask = np.logical(tensor, random >= 0.8)
    train_mask = np.logical(tensor, random < 0.8)
    return val_mask, train_mask

In [ ]:
from torch_geometric.data import Data

train_mask = ratio_bool_switch(data["Set"] != "Validation")
val_mask = ratio_bool_switch(data["Set"] != "Validation")
test_mask = data["Set"] == "Validation"
graph = Data()

In [ ]:
train_mask.sum(), val_mask.sum(), test_mask.sum()

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        conv_1= GATConv(input_dim, hidden_dim,heads=1, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = self.conv_1(x, edge_index).relu()
        x = self.conv_end(x, edge_index)
        return x
            
    def predict_proba(self, graph, mask):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(graph)
            scores = torch.sigmoid(torch.squeeze(logits[mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, graph, mask):
        return torch.round(self.predict_proba(graph, mask)[:, 1])

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

graph = graph.to(device)
WEIGHT = torch.tensor([664])
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

In [ ]:
import torch 

class ModelWrapper():
    def __init__(self, graph):
        self.LEARNING_RATE = 3e-4
        self.MAX_EPOCHS = 40000

        self.model = GraphNeuralNetwork(hidden_dim = 128, out_channels=1) 
        self.model = self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        self.graph = graph
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 10    
        self.val_loss = []
        self.train_loss = []
    
    def validate(self):
        with torch.inference_mode():
            self.model.eval()
            out = self.model(self.graph)
            loss = F.binary_cross_entropy_with_logits(torch.squeeze(out[val_mask]), self.graph.y[val_mask].type(torch.float32),
                                                      pos_weight=WEIGHT)
            self.val_loss.append(loss.item())
            if loss.item() > self.last_loss:
                self.increased_loss += 1
            else:
                self.increased_loss = 0
            self.last_loss = loss.item()

    def train(self):
        for epoch in range(self.MAX_EPOCHS):
            self.model.train()
            self.optimizer.zero_grad()
            out = self.model(self.graph)
            loss = F.binary_cross_entropy_with_logits(torch.squeeze(out[train_mask]), self.graph.y[train_mask].type(torch.float32),
                                                      pos_weight=WEIGHT)
            self.train_loss.append(loss.item())
            loss.backward()
            self.optimizer.step()
            self.validate() 

            if self.increased_loss >= self.BREAKING_THRESHOLD:
#                 print(f"Breaked at {str(epoch)}")
                break
            
    def get_model(self):
        return self.model    
    
    def plot_loss(self):
        plt.plot(self.epochs, self.train_loss, 'g', label='Training loss')
        plt.plot(self.epochs, self.val_loss, 'y', label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()